#### Import Standard Python Modules

In [1]:
import os, time, warnings, sys, json
import joblib
import datetime as dt
import numpy as np
import pandas as pd
warnings.filterwarnings(action="ignore")

#### Import Azure ML SDK Modules

In [2]:
import azureml.core
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies
print(f"Check Azure ML SDK: {azureml.core.VERSION}")

Check Azure ML SDK: 1.16.0


#### Create Azure ML Workspace

In [3]:
with open("configAzureML-bank.json") as f:
    creds = json.load(f)

In [4]:
ws = Workspace.create(name="SALARY_DEC",
                     subscription_id = creds['subscription_id'],
                     resource_group = "abML_DEC",
                     create_resource_group=True,
                     location="westeurope")

Deploying AppInsights with name salarydeinsights89e54808.
Deployed AppInsights with name salarydeinsights89e54808. Took 3.88 seconds.
Deploying KeyVault with name salarydekeyvault6be499b3.
Deploying StorageAccount with name salarydestorage160ca0769.
Deployed KeyVault with name salarydekeyvault6be499b3. Took 18.83 seconds.
Deployed StorageAccount with name salarydestorage160ca0769. Took 22.34 seconds.
Deploying Workspace with name SALARY_DEC.
Deployed Workspace with name SALARY_DEC. Took 21.23 seconds.


In [5]:
os.getcwd()

'C:\\Users\\barth\\Data\\DataBases\\Tutorial_Azure_ML'

In [6]:
path = os.getcwd()
filename = "config_salaryDEC.json"
ws.write_config(path=path, file_name=filename)

#### Access Workspace from config-File

In [7]:
ws_config_path = 'C:\\Users\\barth\Data\DataBases\\Tutorial_Azure_ML\\.azureml\\'
ws_config_file = 'config_salaryDEC.json'
ws = Workspace.from_config(path=ws_config_path, _file_name=ws_config_file)
ws.get_details()

{'id': '/subscriptions/b058cdd0-66ef-48e2-a354-d592fdae4198/resourceGroups/abML_DEC/providers/Microsoft.MachineLearningServices/workspaces/SALARY_DEC',
 'name': 'SALARY_DEC',
 'location': 'westeurope',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'sku': 'Basic',
 'workspaceid': 'c919d8d4-b35b-4528-ab64-7598c9a0357e',
 'description': '',
 'friendlyName': 'SALARY_DEC',
 'creationTime': '2020-12-09T18:22:47.8654787+00:00',
 'keyVault': '/subscriptions/b058cdd0-66ef-48e2-a354-d592fdae4198/resourcegroups/abml_dec/providers/microsoft.keyvault/vaults/salarydekeyvault6be499b3',
 'applicationInsights': '/subscriptions/b058cdd0-66ef-48e2-a354-d592fdae4198/resourcegroups/abml_dec/providers/microsoft.insights/components/salarydeinsights89e54808',
 'identityPrincipalId': '60d14a1c-0e9d-403e-98bf-3cca08ec147e',
 'identityTenantId': 'bd9c9b50-bfed-4bc9-9e1b-7c3fdf883433',
 'identityType': 'SystemAssigned',
 'storageAccount': '/subscriptions/b058cdd0-66ef-48e2-a354-d592fdae4198/resourcegr

#### Create Azure ML Experiment

In [ ]:
exp = Experiment(workspace=ws, name="salexp")

#### Start logging metrics

In [ ]:
run = exp.start_logging()
run.log("Experiment start time:", str(datetime.datetime.now()))

#### Load salary dataset and train the LM model

In [8]:
sal = pd.read_csv('survey_results_public.csv')
sal = sal.iloc[:,[3,5,6, 58,59]]

sal.CompTotal = sal.CompTotal.fillna(sal.CompTotal.median())
sal.Age = sal.Age.fillna(sal.Age.median())
sal.WorkWeekHrs = sal.WorkWeekHrs.fillna(sal.WorkWeekHrs.median())
sal = sal.loc[sal.CompTotal < 90_000, :]

X = sal.Age.values.reshape(-1, 1)
y = sal.CompTotal.values

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=42)

lm = LinearRegression()
lm.fit(X_train, y_train)

print(f"Intercept: {lm.intercept_}")
print(f"Slope: {lm.coef_[0]}")

Intercept: 53587.606708923035
Slope: -98.69396755367482


#### Freeze the LM model (persisting in disk)

In [9]:
fn_output = 'outputs/sal_model.pkl'
joblib.dump(lm,fn_output)

['outputs/sal_model.pkl']

#### Test the LM model

In [10]:
fn = 'outputs/sal_model.pkl'
loaded_model = joblib.load(fn)
y = loaded_model.predict([[45]])[0]
print(y)

49146.378169007665


#### Log metrics to Azure ML Experiment

In [ ]:
run.log("Intercept: ", lm.intercept_)
run.log("Slope: ", lm.coef_[0])

In [ ]:
#! flask/bin/python

#### Register the model

In [11]:
model = Model.register(model_path = 'outputs/sal_model.pkl',
                       model_name = 'sal_model_DEC',
                       tags = {"key":"1", "Superuser":"Andreas"},
                       description = "Salary Prediction LM -V -December2020",
                       workspace = ws
                      )

Registering model sal_model_DEC


#### Define Azure ML Deployment configuration

In [12]:
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               memory_gb=1,
                                               tags={"data": "Salary2",
                                                     "method": "sklearn LM",
                                                     "ProductOwner":"Andreas Barth"},
                                               description = "Predict Stackoverflow Salary"
                                              )

#### Write separate scoring-script => score.py

In [15]:
%%writefile score3.py


import json, os, pickle
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.linear_model import LinearRegression

from azureml.core.model import Model

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = Model.get_model_path('sal_model_DEC')
    model = joblib.load(model_path)
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score3.py


#### Create Environment Configuration file

In [17]:
salenv = CondaDependencies()
salenv.add_conda_package("scikit-learn")

with open("salenv.yml", "w") as f:
    f.write(salenv.serialize_to_string())
    
with open ("salenv.yml", "r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will

# be automatically provisioned for runs with userManagedDependencies=False.


# Details about the Conda environment file format:

# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually


name: project_environment
dependencies:
  # The python interpreter version.

  # Currently Azure ML only supports 3.5.2 and later.

- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.

  - azureml-defaults

- scikit-learn
channels:
- anaconda
- conda-forge



#### Deploy the model to Azure Container Instance / Create Docker Image

In [20]:
image_config = ContainerImage.image_configuration(execution_script = "score3.py",
                                                  runtime = "python",
                                                  conda_file = "salenv.yml", 
                                                  description = "Deployment Dec 2020"
                                                 )

#### Expose web service

In [22]:
service = Webservice.deploy_from_model(workspace = ws,
                                       name = "salary-api_2",
                                       deployment_config = aciconfig,
                                       models = [model],
                                       image_config = image_config)

service.wait_for_deployment(show_output=True)

WebserviceException: WebserviceException:
	Message: Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error, provided service name is invalid. It must only consist of lowercase letters, numbers, or dashes, start with a letter, end with a letter or number, and be between 3 and 32 characters long."
    }
}

#### Get the Web Service URL

[Scoring-Link 31-10-2020](http://1d0016fd-35db-4b22-b18b-e9e4b25657f6.westus.azurecontainer.io/score)

[Scoring-Link 9-12-2020]( )

In [ ]:
print(service.scoring_uri)

In [ ]:
!curl -X POST \
    -H 'Content-Type':'aplication/json' \
    -d '{"data": [[45]]}' \
     http://e48fac8d-c927-4abc-9f58-e484631ca446.westus.azurecontainer.io/score

#### Get the Web Service URL

In [ ]:
print(service.scoring_uri)

In [ ]:
http://e48fac8d-c927-4abc-9f58-e484631ca446.westus.azurecontainer.io/score